In [2]:
# ============================================================================
# 1. KURULUM VE BAĞIMLILIKLAR (DÜZELTİLMİŞ)
# ============================================================================

print("=" * 70)
print("ADIM 1: Gerekli kütüphaneler kuruluyor (Uyumlu sürümlerle)...")
print("=" * 70)

# Önce numpy'yi downgrade et (çakışmaların kaynağı)
!pip install numpy==1.26.4 --force-reinstall -q

# LangChain Core ve Google Genai (pin'lenmiş sürümler)
!pip install langchain-core==0.3.79 langchain==0.3.27 langchain-community==0.3.31 langchain-google-genai==2.1.12 --force-reinstall -q

# Yeni HuggingFace Embeddings paketi (deprecated'i düzeltir)
!pip install langchain-huggingface==0.1.0 -q

# PDF işleme
!pip install pypdf -q

# Embedding için Sentence Transformers (yeniden yükle, numpy sonrası)
!pip install sentence-transformers==3.1.1 -q  # Pin'lenmiş stabil sürüm

# Vector Database için FAISS
!pip install faiss-cpu -q

# Temizlik: Cache'leri temizle
import sys
sys.modules.pop('sentence_transformers', None)  # Eski import'u temizle

print("✅ Tüm kütüphaneler başarıyla kuruldu (Numpy downgrade ile)!\n")

ADIM 1: Gerekli kütüphaneler kuruluyor (Uyumlu sürümlerle)...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.0 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
google-adk 1.17.0 requires tenacity<9.0.0,>=8.0

In [3]:
# ============================================================================
# 2. VERSİYON KONTROLÜ VE API KEY
# ============================================================================

print("=" * 70)
print("ADIM 2: Versiyon kontrolü ve API anahtarı yükleniyor...")
print("=" * 70)

import os
from google.colab import userdata
import langchain

# Versiyon bilgileri
print(f"📦 LangChain: {langchain.__version__}")
!pip show langchain-google-genai | grep Version
!pip show langchain-core | grep Version

# API Key yükleme
try:
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    print("\n✅ Google API Anahtarı başarıyla yüklendi.")
except:
    print("\n⚠️ UYARI: 'GOOGLE_API_KEY' Colab Secrets'e eklenmedi!")
    print("Sol menüden 🔑 Secrets bölümüne gidin ve GOOGLE_API_KEY ekleyin.")

print()

ADIM 2: Versiyon kontrolü ve API anahtarı yükleniyor...
📦 LangChain: 0.3.27
Version: 2.1.12
Version: 0.3.79

✅ Google API Anahtarı başarıyla yüklendi.



In [4]:
# ============================================================================
# 3. GEREKLİ MODÜLLER İÇE AKTARILIYOR (DÜZELTİLMİŞ)
# ============================================================================

print("=" * 70)
print("ADIM 3: Modüller içe aktarılıyor (Güncellenmiş import'lar)...")
print("=" * 70)

try:
    from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
    from langchain_community.document_loaders import PyPDFLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain_community.vectorstores import FAISS
    from langchain.chains import RetrievalQA
    # DÜZELTME: Eski import yerine yenisini kullan
    from langchain_huggingface import HuggingFaceEmbeddings  # Yeni paket
    print("✅ Tüm modüller başarıyla içe aktarıldı!\n")
except Exception as e:
    print(f"❌ Modül içe aktarma hatası: {e}\n")
    raise

ADIM 3: Modüller içe aktarılıyor (Güncellenmiş import'lar)...
✅ Tüm modüller başarıyla içe aktarıldı!



In [5]:
# ============================================================================
# 4. VERİ SETİ YÜKLEME VE İŞLEME
# ============================================================================

print("=" * 70)
print("ADIM 4: Veri Seti (PDF) Yükleniyor ve İşleniyor...")
print("=" * 70)

"""
VERİ SETİ HAKKINDA:
- Dosya Adı: sozluk.pdf
- İçerik: Ekonomi terimleri sözlüğü
- Format: PDF
- Kaynak: [Buraya veri kaynağını ekleyin]
- Toplam Sayfa: [PDF yüklendikten sonra görülecek]

NOT: Bu dosyayı Colab'ın sol menüsündeki "Files" bölümünden
/content/ dizinine yüklemeniz gerekmektedir.
"""

pdf_path = "/content/sozluk.pdf"

try:
    # PDF Yükleme
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    print(f"✅ PDF başarıyla yüklendi")
    print(f"📄 Toplam sayfa sayısı: {len(docs)}")
    print(f"📊 İlk sayfadaki karakter sayısı: {len(docs[0].page_content)}")

    # Örnek içerik gösterimi
    print("\n📖 İlk sayfadan örnek içerik (ilk 200 karakter):")
    print("-" * 70)
    print(docs[0].page_content[:200] + "...")
    print("-" * 70)
    print()

except FileNotFoundError:
    print(f"❌ HATA: '{pdf_path}' dosyası bulunamadı!")
    print("📁 Lütfen PDF'i Colab'ın sol menüsünden /content/ dizinine yükleyin.")
    raise
except Exception as e:
    print(f"❌ PDF yükleme hatası: {e}")
    raise

ADIM 4: Veri Seti (PDF) Yükleniyor ve İşleniyor...
✅ PDF başarıyla yüklendi
📄 Toplam sayfa sayısı: 210
📊 İlk sayfadaki karakter sayısı: 0

📖 İlk sayfadan örnek içerik (ilk 200 karakter):
----------------------------------------------------------------------
...
----------------------------------------------------------------------



In [6]:
# ============================================================================
# 5. METİN PARÇALAMA (CHUNKING)
# ============================================================================

print("=" * 70)
print("ADIM 5: Metin Parçalara Ayrılıyor (Chunking)...")
print("=" * 70)

"""
CHUNKING STRATEJİSİ:
- Chunk Size: 1000 karakter (Her parça ~200-250 kelime)
- Chunk Overlap: 100 karakter (Bağlam kaybını önlemek için)
- Metod: RecursiveCharacterTextSplitter (Hiyerarşik bölme)

NEDEN BU PARAMETRELER?
- 1000 karakter: Embedding modelleri için optimal boyut
- 100 overlap: Terimlerin parçalar arası kaybolmaması için
- Recursive splitter: Cümle yapısını koruyarak böler
"""

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)

chunks = text_splitter.split_documents(docs)

print(f"✅ Doküman {len(chunks)} parçaya (chunk) bölündü")
print(f"📊 Ortalama chunk boyutu: {sum(len(c.page_content) for c in chunks) // len(chunks)} karakter")
print(f"\n📝 Örnek bir chunk:")
print("-" * 70)
print(chunks[0].page_content[:300] + "...")
print("-" * 70)
print()

ADIM 5: Metin Parçalara Ayrılıyor (Chunking)...
✅ Doküman 383 parçaya (chunk) bölündü
📊 Ortalama chunk boyutu: 755 karakter

📝 Örnek bir chunk:
----------------------------------------------------------------------
1
a
e
c,ç
ı,i
o,öp,r
m,n
s,ş
u,ü
v,y,z
l
g
b
k
f
d
h
t
a
e
c,ç
i
o,ö
p,r
s
u,ü
v,y
m,n
g
b
k,l
f
d
h
t...
----------------------------------------------------------------------



In [7]:
# ============================================================================
# 6. EMBEDDING MODELİ VE VECTOR DATABASE (DÜZELTİLMİŞ)
# ============================================================================

print("=" * 70)
print("ADIM 6: Embedding Modeli Yükleniyor ve Vector DB Oluşturuluyor...")
print("=" * 70)

try:
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

    print(f"🤖 Embedding modeli yükleniyor: {model_name}")
    # DÜZELTME: Yeni import ile HuggingFaceEmbeddings kullan
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    print("✅ Embedding modeli hazır")

    print("\n🔍 Vector Database (FAISS) oluşturuluyor...")
    print("⏳ Bu işlem birkaç dakika sürebilir...")

    vector_store = FAISS.from_documents(chunks, embeddings)

    print(f"✅ FAISS Vector Database başarıyla oluşturuldu!")
    print(f"📊 Toplam vektör sayısı: {len(chunks)}")
    print()

except Exception as e:
    print(f"❌ Embedding veya Vector DB hatası: {e}")
    raise

ADIM 6: Embedding Modeli Yükleniyor ve Vector DB Oluşturuluyor...
🤖 Embedding modeli yükleniyor: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Embedding modeli hazır

🔍 Vector Database (FAISS) oluşturuluyor...
⏳ Bu işlem birkaç dakika sürebilir...
✅ FAISS Vector Database başarıyla oluşturuldu!
📊 Toplam vektör sayısı: 383



In [8]:
# ============================================================================
# 7. RAG CHAIN (CHATBOT BEYNİ) KURULUMU
# ============================================================================

print("=" * 70)
print("ADIM 7: RAG Chain (Retrieval Augmented Generation) Kuruluyor...")
print("=" * 70)

"""
RAG MİMARİSİ AÇIKLAMASI:

1. RETRIEVAL (Bilgi Getirme):
   - Kullanıcı sorusu embedding'e çevriliyor
   - Vector DB'de similarity search yapılıyor
   - En alakalı k=10 chunk getiriliyor

2. AUGMENTATION (Bağlam Zenginleştirme):
   - Getirilen chunk'lar birleştiriliyor
   - LLM'e context olarak veriliyor

3. GENERATION (Cevap Üretme):
   - Gemini 2.5 Flash context'i okuyup cevap üretiyor
   - Temperature=0.1 (tutarlı cevaplar için)

NEDEN RAG?
- LLM'in bilgisi sınırlı (training cutoff)
- PDF'deki güncel/spesifik bilgiyi kullanabiliyor
- Hallusinasyon riski azalıyor (kaynak var)
"""

try:
    # LLM Konfigürasyonu
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",  # En güncel Gemini modeli
        temperature=0.1,  # Düşük = daha tutarlı/gerçekçi cevaplar
    )

    # Retriever Konfigürasyonu
    retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 10}  # En benzer 10 chunk getir
    )

    # QA Chain Oluşturma
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Tüm context'i tek seferde LLM'e ver
        retriever=retriever,
        return_source_documents=True  # Kaynak chunk'ları da döndür
    )

    print("✅ RAG Chain başarıyla kuruldu!")
    print(f"🤖 LLM: Gemini 2.0 Flash")
    print(f"🔍 Retriever: FAISS (k=10)")
    print(f"🌡️ Temperature: 0.1")
    print()

except Exception as e:
    print(f"❌ RAG Chain kurulum hatası: {e}")
    qa_chain = None
    raise

ADIM 7: RAG Chain (Retrieval Augmented Generation) Kuruluyor...
✅ RAG Chain başarıyla kuruldu!
🤖 LLM: Gemini 2.0 Flash
🔍 Retriever: FAISS (k=10)
🌡️ Temperature: 0.1



In [9]:
# ============================================================================
# 8. CHATBOT FONKSİYONU
# ============================================================================

def ask_question(question, show_sources=False):
    """
    Chatbot'a soru sorma fonksiyonu

    Args:
        question (str): Kullanıcının sorusu
        show_sources (bool): Kaynak chunk'ları göster/gösterme

    Returns:
        dict: Cevap ve kaynak bilgileri
    """
    print("\n" + "=" * 70)
    print(f"❓ SORU: {question}")
    print("=" * 70)

    if qa_chain is None:
        print("❌ RAG Chain tanımlı değil!")
        return None

    try:
        # Soru-cevap işlemi
        result = qa_chain.invoke({"query": question})

        print(f"\n💬 CEVAP:")
        print("-" * 70)
        print(result['result'])
        print("-" * 70)

        # Kaynak dökümanları göster
        if show_sources and 'source_documents' in result:
            print(f"\n📚 KULLANILAN KAYNAKLAR ({len(result['source_documents'])} adet):")
            for i, doc in enumerate(result['source_documents'][:3], 1):  # İlk 3'ü göster
                print(f"\n[Kaynak {i}] Sayfa {doc.metadata.get('page', 'N/A')}:")
                print(doc.page_content[:200] + "...")

        return result

    except Exception as e:
        print(f"❌ Soru işleme hatası: {e}")
        return None

In [10]:
# ============================================================================
# 9. TEST SORULARI VE DEĞERLENDİRME
# ============================================================================

print("=" * 70)
print("ADIM 9: Test Soruları ile Chatbot Deneniyor...")
print("=" * 70)

# Test soruları
test_questions = [
    "Arbitraj nedir?",
    "Enflasyon kelimesinin anlamını açıklar mısın?",
    "Deflasyon nedir?",
    "Merkez bankası ne iş yapar?",
    "Devalüasyon ile revalüasyon arasındaki fark nedir?"
]

print("\n🧪 5 farklı ekonomi terimi ile test ediliyor...\n")

results = []
for i, question in enumerate(test_questions, 1):
    print(f"\n{'#' * 70}")
    print(f"TEST {i}/{len(test_questions)}")
    print(f"{'#' * 70}")

    result = ask_question(question, show_sources=True)
    results.append({
        'question': question,
        'answer': result['result'] if result else None,
        'success': result is not None
    })

    print("\n" + "⏳ " * 20)

ADIM 9: Test Soruları ile Chatbot Deneniyor...

🧪 5 farklı ekonomi terimi ile test ediliyor...


######################################################################
TEST 1/5
######################################################################

❓ SORU: Arbitraj nedir?



💬 CEVAP:
----------------------------------------------------------------------
Arbitraj: Herhangi bir menkul kıymetin, dövizin veya malın eş anlı olarak bir piyasadan alınıp diğer bir piyasada satılarak kıymetler birbiri ile değiştirilmesidir.
----------------------------------------------------------------------

📚 KULLANILAN KAYNAKLAR (10 adet):

[Kaynak 1] Sayfa 11:
lara bölünmüş olan, borçlarıyla yükümleri yalnız or-
taklığın mal varlığı ile karşılanan ve her ortağın or -
taklığa karşı sorumluluğu, yalnız kendi sermayesi ile 
sınırlı bulunan ticaret ortaklığıdır...

[Kaynak 2] Sayfa 66:
66
Ekonomi Terimleri Sözlüğü
Londra bankalar arası işlemler faiz oranı referans ola-
rak kullanılır. 
Likidite: Kolaylıkla paraya çevrilebilecek özellikte
ki varlıklardır. 
Likidite tuzağı: Para arzı ...

[Kaynak 3] Sayfa 182:
182
Ekonomi Terimleri Sözlüğü
Toplu pazarlık: Toplu iş sözleşmesi yapmak üzere 
yetkili tarafların resmen bir araya gelerek toplu iş söz-
leşmesinin şartları üzerinde görüşm

In [11]:
# ============================================================================
# 10. SONUÇLAR VE İSTATİSTİKLER
# ============================================================================

print("\n" + "=" * 70)
print("📊 TEST SONUÇLARI ve İSTATİSTİKLER")
print("=" * 70)

successful = sum(1 for r in results if r['success'])
print(f"\n✅ Başarılı sorgu: {successful}/{len(results)}")
print(f"📈 Başarı oranı: {(successful/len(results)*100):.1f}%")

print("\n💡 PROJE KAZANIMLARI:")
print("  ✓ PDF'den bilgi çekme")
print("  ✓ Semantic search ile ilgili parçaları bulma")
print("  ✓ LLM ile doğal dil cevapları")
print("  ✓ Türkçe dil desteği")
print("  ✓ Kaynak referansları")

print("\n" + "=" * 70)
print("🎉 PROJE BAŞARIYLA TAMAMLANDI!")
print("=" * 70)

print("\n📝 SONRAKI ADIMLAR:")
print("1. Web arayüzü geliştirin (Streamlit/Gradio)")
print("2. README.md dosyasını oluşturun")
print("3. requirements.txt hazırlayın")
print("4. GitHub'a yükleyin")
print("5. Deploy edin (Hugging Face Spaces/Streamlit Cloud)")
print("\n" + "=" * 70)


📊 TEST SONUÇLARI ve İSTATİSTİKLER

✅ Başarılı sorgu: 5/5
📈 Başarı oranı: 100.0%

💡 PROJE KAZANIMLARI:
  ✓ PDF'den bilgi çekme
  ✓ Semantic search ile ilgili parçaları bulma
  ✓ LLM ile doğal dil cevapları
  ✓ Türkçe dil desteği
  ✓ Kaynak referansları

🎉 PROJE BAŞARIYLA TAMAMLANDI!

📝 SONRAKI ADIMLAR:
1. Web arayüzü geliştirin (Streamlit/Gradio)
2. README.md dosyasını oluşturun
3. requirements.txt hazırlayın
4. GitHub'a yükleyin
5. Deploy edin (Hugging Face Spaces/Streamlit Cloud)



In [17]:
# requirements.txt oluştur/güncelle
requirements_content = """
langchain==0.3.27
langchain-community==0.3.31
langchain-google-genai==2.1.12
langchain-huggingface==0.1.0
sentence-transformers==3.1.1
faiss-cpu
pypdf
protobuf==4.25.5
numpy==1.26.4
streamlit
google-generativeai
"""

with open('/content/requirements.txt', 'w') as f:
    f.write(requirements_content.strip())

print("✅ requirements.txt başarıyla oluşturuldu/güncellendi!")
print("📄 İçerik:")
with open('/content/requirements.txt', 'r') as f:
    print(f.read())

✅ requirements.txt başarıyla oluşturuldu/güncellendi!
📄 İçerik:
langchain==0.3.27
langchain-community==0.3.31
langchain-google-genai==2.1.12
langchain-huggingface==0.1.0
sentence-transformers==3.1.1
faiss-cpu
pypdf
protobuf==4.25.5
numpy==1.26.4
streamlit
google-generativeai


In [18]:
# ============================================================================
# 11. GRADIO WEB ARAYÜZÜ
# ============================================================================

print("\n" + "=" * 70)
print("ADIM 11: Gradio Web Arayüzü Başlatılıyor...")
print("=" * 70)

# Gradio kurulumu
!pip install gradio -q

import gradio as gr
from datetime import datetime

def chatbot_interface(message, history):
    """
    Gradio chatbot fonksiyonu

    Args:
        message (str): Kullanıcının mesajı
        history (list): Sohbet geçmişi (messages format)

    Returns:
        tuple: (Boş string, güncellenmiş history)
    """
    if not message.strip():
        return "", history

    try:
        # RAG chain ile cevap al
        result = qa_chain.invoke({"query": message})
        answer = result['result']

        # Kaynak bilgisi ekle (isteğe bağlı)
        if 'source_documents' in result and len(result['source_documents']) > 0:
            sources = set()
            for doc in result['source_documents'][:3]:
                page = doc.metadata.get('page', 'N/A')
                sources.add(f"Sayfa {page}")

            if sources:
                answer += f"\n\n📚 *Kaynaklar: {', '.join(sorted(sources))}*"

        # History'ye ekle (messages formatında - dictionary)
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": answer})
        return "", history

    except Exception as e:
        error_msg = f"❌ Bir hata oluştu: {str(e)}"
        history.append({"role": "user", "content": message})
        history.append({"role": "assistant", "content": error_msg})
        return "", history

# Gradio arayüzü tasarımı
with gr.Blocks(
    theme=gr.themes.Soft(),
    title="📚 Ekonomi Terimleri Chatbot"
) as demo:

    gr.Markdown(
        """
        # 📚 Ekonomi Terimleri RAG Chatbot
        ### Powered by Google Gemini 2.0 Flash & LangChain

        Bu chatbot, yüklediğiniz ekonomi sözlüğü PDF'inden bilgi çekerek sorularınızı yanıtlar.
        **RAG (Retrieval Augmented Generation)** teknolojisi kullanır.
        """
    )

    with gr.Row():
        with gr.Column(scale=2):
            # Chatbot arayüzü
            chatbot = gr.Chatbot(
                label="Sohbet",
                height=500,
                bubble_full_width=False,
                avatar_images=(None, "🤖"),
                type="messages"  # Modern format
            )

            with gr.Row():
                msg = gr.Textbox(
                    label="Sorunuzu yazın",
                    placeholder="Örn: Arbitraj nedir?",
                    scale=4,
                    lines=1
                )
                submit = gr.Button("📤 Gönder", variant="primary", scale=1)

            with gr.Row():
                clear = gr.Button("🗑️ Sohbeti Temizle")

        with gr.Column(scale=1):
            gr.Markdown(
                """
                ### 💡 Örnek Sorular

                - Arbitraj nedir?
                - Enflasyon ne demek?
                - Deflasyon ile enflasyon farkı nedir?
                - Merkez bankası ne iş yapar?
                - Devalüasyon açıkla
                - Revalüasyon nedir?
                - Döviz kuru nasıl belirlenir?

                ### ℹ️ Bilgi

                **Model:** Gemini 2.0 Flash
                **Embedding:** Multilingual MiniLM
                **Vector DB:** FAISS
                **Toplam Chunk:** """ + str(len(chunks)) + """
                **Retrieval:** Top-10 Similarity

                ### 📊 İstatistikler

                - ✅ RAG Chain Aktif
                - 🔍 Semantic Search Etkin
                - 🌐 Türkçe Destek
                - 📄 PDF Kaynak Referansı
                """
            )

    gr.Markdown(
        """
        ---
        <center>

        Made with ❤️ using LangChain & Gradio |
        [GitHub](https://github.com) |
        [Documentation](https://docs.langchain.com)

        </center>
        """
    )

    # Event handlers
    submit.click(
        fn=chatbot_interface,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot]
    )

    msg.submit(
        fn=chatbot_interface,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot]
    )

    clear.click(
        fn=lambda: ([], ""),
        inputs=None,
        outputs=[chatbot, msg]
    )

# Arayüzü başlat
print("\n✅ Gradio arayüzü hazır!")
print("🌐 Aşağıdaki linkten erişebilirsiniz...\n")

# Public link ile başlat (Colab'da çalışması için)
demo.launch(
    share=True,  # Public link oluştur
    debug=True,
    show_error=True
)

print("\n" + "=" * 70)
print("🎉 WEB ARAYÜZÜ BAŞARIYLA BAŞLATILDI!")
print("=" * 70)
print("\n💡 İPUCU: Gradio arayüzünü kapatmak için 'Kernel > Interrupt' yapın.")


ADIM 11: Gradio Web Arayüzü Başlatılıyor...


/tmp/ipython-input-3979755205.py:74: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(



✅ Gradio arayüzü hazır!
🌐 Aşağıdaki linkten erişebilirsiniz...

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c5faab4f1e88723e72.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://c5faab4f1e88723e72.gradio.live

🎉 WEB ARAYÜZÜ BAŞARIYLA BAŞLATILDI!

💡 İPUCU: Gradio arayüzünü kapatmak için 'Kernel > Interrupt' yapın.
